In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark DataFrames").getOrCreate()

In [0]:
data = [("Manali", "IT", 50000), ("Stuti", "HR", 45000), ("Vaishnav", "IT", 60000)]
cols = ["Name", "Department", "Salary"]

df=spark.createDataFrame(data,cols)
display(df)

In [0]:
df.write.format("delta").mode("overwrite").saveAsTable("employee_delta")

In [0]:
%sql
select * from employee_delta

In [0]:
%sql
update employee_delta set salary = 100000 where name = 'Manali'

In [0]:
%sql
describe history employee_delta

In [0]:
# MERGE INTO - update a row
from pyspark.sql import Row

update_data = [Row(Name="Alice", Department="IT", Salary=52000)]
df_update = spark.createDataFrame(update_data)
df_update.createOrReplaceTempView("updates_view")


spark.sql("""
MERGE INTO employee_delta AS target
USING updates_view AS source
ON target.Name = source.Name
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *
""")

In [0]:
%sql
describe history employee_delta

In [0]:
#Schema Evolution (Add New Column)

new_data = [("David", "Finance", 70000, 5)]  # 5 years experience
cols_new = ["Name", "Department", "Salary", "Experience"]
df_new=spark.createDataFrame(new_data,cols_new) 
display(df_new)


#Merge Schema 
df_new.write.format("delta") \
    .option("mergeSchema", "true") \
    .mode("append") \
    .saveAsTable("employee_delta")

In [0]:
%sql
describe extended employee_delta

In [0]:
%sql
select * from employee_delta